In [1]:
import sys
import pandas as pd
import matplotlib.pyplot as plt

# Add parent directory to path to allow import of config.py
sys.path.append("..")

import config as conf

ModuleNotFoundError: No module named 'matplotlib'

First step will be to load, parse, format and clean the raw datafiles we exported from the LMC enrollment tableau (link). We have two files, one where all metrics are aggregated college wide, and a second where the data is split by department.

In [ ]:
enrollment_by_semester = pd.read_csv(f'{conf.DATA_PATH}{conf.ENROLLMENT_BY_SEMESTER_DATAFILE}')
enrollment_by_department = pd.read_csv(f'{conf.DATA_PATH}{conf.ENROLLMENT_BY_DEPARTMENT_DATAFILE}')

In [ ]:
enrollment_by_semester.head()

In [ ]:
enrollment_by_department.head()

Looks like there are some garb are some garbage columns we need to get rid of. Also, the data appears to be in R-style long format so we should reformat it to wide so we can handle it more like a canonical Pandas dataframe.

In [ ]:
# Drop columns we don't need
enrollment_by_semester.drop(['Level 2', 'Level 3', 'NegPos_Number', 'List of Values'], axis=1, inplace=True)
enrollment_by_department.drop(['Level 3', 'NegPos_Number', 'List of Values'], axis=1, inplace=True)

In [ ]:
# Reformat dataframe from wide to long
enrollment_by_semester = enrollment_by_semester.pivot_table(index='Level 1', columns='Measure Names', values='Measure Values').reset_index()
enrollment_by_department = enrollment_by_department.pivot_table(index=['Level 1','Level 2'], columns='Measure Names', values='Measure Values').reset_index()

In [ ]:
# Split 'Level 1' column string into year and semester, add back to dataframe as seperate columns

term = enrollment_by_semester['Level 1'].str.extract('(?P<year>\d{4})(?P<semester>[a-zA-Z]{2})', expand=True)
enrollment_by_semester = pd.concat([enrollment_by_semester, term], axis=1)

term = enrollment_by_department['Level 1'].str.extract('(?P<year>\d{4})(?P<semester>[a-zA-Z]{2})', expand=True)
enrollment_by_department = pd.concat([enrollment_by_department, term], axis=1)

In [ ]:
enrollment_by_semester.describe()

In [ ]:
enrollment_by_department.describe()

In [ ]:
plt.matshow(enrollment_by_semester.corr())
plt.show()